# 自然语言情感分析

人类自然语言中包含了丰富的情感色彩，情感倾向分析（Sentiment Classification）即针对带有主观描述的中文文本，自动判断该文本的情感极性类别并给出相应的置信度。情感倾向分析能够帮助企业理解用户消费习惯、分析热点话题和危机舆情监控，为企业提供有利的决策支持。，利用机器自动分析这些情感倾向，不但有助于帮助企业了解消费者对其产品的感受，为产品改进提供依据；同时还有助于企业分析商业伙伴们的态度，以便更好地进行商业决策。

简单的说，我们可以将情感分析（sentiment classification）任务定义为一个分类问题，即指定一个文本输入，机器通过对文本进行分析、处理、归纳和推理后自动输出结论，如**图1**所示。
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/0a5a60199b0d49daaf6e1dfe616accce7a0c97f8c5ad4ad2b9bf0013d2c3ecab" width="800" ></center>
<br><center>图1：情感分析任务</center></br>

通常情况下，人们把情感分析任务看成一个三分类问题，如 **图2** 所示：
<br></br>
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/b630901b397e4e7a8e78ab1d306dfa1fc070d91015a64ef0b8d590aaa8cfde14" width="600" ></center>
<br><center>图2：情感分析任务</center></br>

- **正向：** 表示正面积极的情感，如高兴，幸福，惊喜，期待等。
- **负向：** 表示负面消极的情感，如难过，伤心，愤怒，惊恐等。
- **其他：** 其他类型的情感。

在情感分析任务中，研究人员除了分析句子的情感类型外，还细化到以句子中具体的“方面”为分析主体进行情感分析（aspect-level），如下：

> 这个薯片口味有点咸，太辣了，不过口感很脆。

关于薯片的口味方面是一个负向评价（咸，太辣），然而对于口感方面却是一个正向评价（很脆）。

> 我很喜欢夏威夷，就是这边的海鲜太贵了。

关于夏威夷是一个正向评价（喜欢），然而对于夏威夷的海鲜却是一个负向评价（价格太贵）。




# 使用深度神经网络完成情感分析任务

一般通过深度神经网络将句子表示成向量，接入分类器，从而实现句子分类。

如何把自然语言句子转换向量呢？

有一个非常简单粗暴的解决方式：就是先把一个句子中所有词的embedding进行加和平均，再用得到的平均embedding作为整个句子的向量表示。然而由于自然语言变幻莫测，我们在使用神经网络处理句子的时候，往往会遇到如下两类问题：

- **变长的句子：** 自然语言句子往往是变长的，不同的句子长度可能差别很大。然而大部分神经网络接受的输入都是张量，长度是固定的，那么如何让神经网络处理变长数据成为了一大挑战。

- **组合的语义：** 自然语言句子往往对结构非常敏感，有时稍微颠倒单词的顺序都可能改变这句话的意思，比如：

  >你等一下我做完作业就走。
  >
  >我等一下你做完工作就走。

  >我不爱吃你做的饭。
  >
  >你不爱吃我做的饭。
  
  >我瞅你咋地。
  >
  >你瞅我咋地。
  
因此，我们需要找到一个可以考虑词和词之间顺序（关系）的神经网络，用于更好地实现自然语言句子建模。

循环神经网络(Recurrent Neural Network, RNN)广泛适用于自然语言处理领域(Natural Language Processing, NLP)，RNN有什么显著的特点呢？普通的神经网络，每一层的输出是下一层的输入，每一层之间是相互独立的，没有关系。但是对于语言来说，一句话中的单词顺序不同，整个语义就完全变了。因此自然语言处理往往需要能够更好地处理序列信息的神经网络，RNN 能够满足这个需求。

RNN 中，隐藏层的状态，不仅取决于当前输入层的输出，还和上一步隐藏层的状态有关。

PaddleNLP将常见[RNN类模型](http://gitlab.baidu.com/PaddleSL/PaddleNLP/tree/master/examples/text_classification/rnn)和经典[预训练模型](http://gitlab.baidu.com/PaddleSL/PaddleNLP/tree/master/examples/text_classification/pretrained_models)封装到了[seq2vec](http://gitlab.baidu.com/PaddleSL/PaddleNLP/tree/master/paddlenlp/seq2vec)，便于大家通过配置参数灵活调用。这里采用了LSTM网络。

长短期记忆模型(Long short-term memory, LSTM)是一种特殊的RNN，主要是为了解决长序列训练过程中的梯度消失和梯度爆炸问题。简单来说，就是相比普通的RNN，LSTM能够在更长的序列中有更好的表现。


关于数据集，这里采用的是PaddleNLP内置数据集之一：情感倾向分类数据集ChnSentiCorp。


# 长短时记忆网络LSTM

长短时记忆网络（Long Short-Term Memory，LSTM）是一个非常经典的循环神经网络（Recurrent Neural Network，RNN）可以对自然语言句子或是其它时序信号进行建模，RNN网络结构如 **图5** 所示。
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/3f4d413393824d208177a020dcfa68205fa192ebecf04b42a5a17cbe7a146abb" width="600" ></center>
<br><center>图5：循环神经网络结构</center></br>

普通RNN网络存在一个明显的缺陷，就是当阅读很长的序列时，网络内部的信息会变得越来越复杂，甚至会超过网络的记忆能力，使得最终的输出信息变得混乱无用。长短时记忆网络（Long Short-Term Memory，LSTM）内部的复杂结构正是为处理这类问题而设计的，其网络结构如 **图6** 所示。
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/d7d112d458ff401ba7fb089cf0521b284c795798dcfb4b88bd2dd6b65854193a" width="600" ></center>
<br><center>图6：LSTM网络结构</center></br>

长短时记忆网络的结构和循环神经网络非常类似，都是通过不断调用同一个cell来逐次处理时序信息。每阅读一个新单词$x_t$，就会输出一个新的输出信号$h_t$，用来表示当前阅读到所有内容的整体向量表示。不过二者又有一个明显区别，长短时记忆网络在不同cell之间传递的是两个记忆信息，而不像循环神经网络一样只有一个记忆信息，此外长短时记忆网络的内部结构也更加复杂，如 **图7** 所示。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/024b611db4ad4b709a4e1bd403ac41897c73792b895d4557a2dd561d625fcbf1" width="400" ></center>
<br><center>图7：LSTM网络内部结构示意图</center></br>

区别于循环神经网络RNN，长短时记忆网络最大的特点是在更新内部记忆时，引入了遗忘机制。即容许网络忘记过去阅读过程中看到的一些无关紧要的信息，只保留有用的历史信息。通过这种方式延长了记忆长度。举个例子：

> 我觉得这家餐馆的菜品很不错，烤鸭非常正宗，包子也不错，酱牛肉很有嚼劲。但是服务员态度太恶劣了，我们在门口等了50分钟都没有能成功进去，好不容易进去了，桌子也半天没人打扫。整个环境非常吵闹，我的孩子都被吓哭了，我下次不会带朋友来。

当我们阅读上面这段话的时候，可能会记住一些关键词，如烤鸭好吃、牛肉有嚼劲、环境吵等，但也会忽略一些不重要的内容，如“我觉得”、“好不容易”等，长短时记忆网络正是受这个启发而设计的。

长短时记忆网络的Cell有三个输入：

- 这个网络新看到的输入信号，如下一个单词，记为$x_{t}$， 其中$x_{t}$是一个向量，$t$代表了当前时刻。
- 这个网络在上一步的输出信号，记为$h_{t-1}$，这是一个向量，维度同$x_{t}$相同。
- 这个网络在上一步的记忆信号，记为$c_{t-1}$，这是一个向量，维度同$x_{t}$相同。

得到这两个信号之后，长短时记忆网络没有立即去融合这两个向量，而是计算了权重。

- 输入门：$i_{t}=sigmoid(W_{i}X_{t}+V_{i}H_{t-1}+b_i)$，控制有多少输入信号会被融合。
- 遗忘门：$f_{t}=sigmoid(W_{f}X_{t}+V_{f}H_{t-1}+b_f)$，控制有多少过去的记忆会被遗忘。
- 输出门：$o_{t}=sigmoid(W_{o}X_{t}+V_{o}H_{t-1}+b_o)$，控制最终输出多少记忆。
- 单元状态：$g_{t}=tanh(W_{g}X_{t}+V_{g}H_{t-1}+b_g)$，输入信号和过去的输入信号做一个信息融合。

通过学习这些门的权重设置，长短时记忆网络可以根据当前的输入信号和记忆信息，有选择性地忽略或者强化当前的记忆或是输入信号，帮助网络更好地学习长句子的语义信息：

- 记忆信号：$c_{t} = f_{t} \cdot c_{t-1} + i_{t} \cdot g_{t}$

- 输出信号：$h_{t} = o_{t} \cdot tanh(c_{t})$

# 使用PaddlNLP实现LSTM句子情感分类
## 一、安装

In [1]:
import paddle
paddle.__version__
# !pip install paddlenlp

import paddle
import paddle.nn as nn
import paddle.nn.functional as F

## 二、数据加载和处理

In [2]:
from chnsenticorp import ChnSentiCorp
from data_process import load_vocab, convert_tokens_to_ids
from functools import partial
import jieba
import numpy as np
from dataset import TSVDataset, SimpleDataset

### 下载数据集
- 使用内置数据集：[PaddleNLP内置数据集](http://gitlab.baidu.com/PaddleSL/PaddleNLP/tree/master/paddlenlp/datasets)
- 自定义数据集 <font color=red size=4>待补充自定义数据集的demo</font>

ChnSentiCorp数据集在`paddlenlp.datasets`中是可以直接获取的。   

通过`paddlenlp.datasets`获取到的数据是否已经经过了文本预处理，即 Tokenizer，向量化文本(将文本转为数字序列)。<font color=red size=4>这里待统一</font>

In [4]:
# 加载paddlenlp.datasets的内置数据集
train_dataset = ChnSentiCorp('train')
dev_dataset = ChnSentiCorp('dev')
test_dataset = ChnSentiCorp('test')

100%|██████████| 1713/1713 [00:00<00:00, 11732.69it/s]


<class 'chnsenticorp.ChnSentiCorp'>
['0', '1']
这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般 1


**数据格式**    
每个例子包含一句评论和对应的标签，0或1。0代表负向评论，1代表正向评论。   
看一下前5条数据和对应标签。

In [10]:
print (type(train_dataset))
print (train_dataset.get_labels())

for sent, label in test_dataset[:5]:
    print (sent, label)

<class 'chnsenticorp.ChnSentiCorp'>
['0', '1']
这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般 1
怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片！开始还怀疑是不是赠送的个别现象，可是后来发现每张DVD后面都有！真不知道生产商怎么想的，我想看的是猫和老鼠，不是米老鼠！如果厂家是想赠送的话，那就全套米老鼠和唐老鸭都赠送，只在每张DVD后面添加一集算什么？？简直是画蛇添足！！ 0
还稍微重了点，可能是硬盘大的原故，还要再轻半斤就好了。其他要进一步验证。贴的几种膜气泡较多，用不了多久就要更换了，屏幕膜稍好点，但比没有要强多了。建议配赠几张膜让用用户自己贴。 0
交通方便；环境很好；服务态度很好 房间较小 1
不错，作者的观点很颠覆目前中国父母的教育方式，其实古人们对于教育已经有了很系统的体系了，可是现在的父母以及祖父母们更多的娇惯纵容孩子，放眼看去自私的孩子是大多数，父母觉得自己的孩子在外面只要不吃亏就是好事，完全把古人几千年总结的教育古训抛在的九霄云外。所以推荐准妈妈们可以在等待宝宝降临的时候，好好学习一下，怎么把孩子教育成一个有爱心、有责任心、宽容、大度的人。 1


In [ ]:
# 自定义数据集
# to do

### 数据处理
- **对齐句子长度，用于一个batch内批运算**       
句子长度统一设为多少？可指定固定长度，或选取数据集中最大句子长度。这里采用每个batch中的最大长度。
- **数据多进程异步加载（可选）**      
使用`paddle.io.DataLoader`接口

<font color=red size=4>？collate_fn代码怎么理解；
generate_batch()里遍历batch为啥有长度，各个dataset返回的数据项是否一致；构造batch的逻辑是否能通用？；pad_texts_to_max_seq_len和convert_example与dataloder耦合，可接耦，并作为通用数据处理函数</font>




#### 数据预处理

In [ ]:
# 构造词-id词典
vocab = load_vocab("./word_dict.txt")
print ('%d words' % len(vocab), type(vocab))
for term in list(vocab)[:5]:
    print (term, vocab[term])

# 为特殊符PAD指定id
if '[PAD]' not in vocab:
    pad_token_id = len(vocab)
    vocab['[PAD]'] = pad_token_id
else:
    pad_token_id = vocab['[PAD]']

# 对齐句子长度
def pad_texts_to_max_seq_len(texts, max_seq_len, pad_token_id=0):
    for index, text in enumerate(texts):
        seq_len = len(text)
        if seq_len < max_seq_len:
            padded_tokens = [pad_token_id for _ in range(max_seq_len - seq_len)]
            new_text = text + padded_tokens
            texts[index] = new_text
        elif seq_len > max_seq_len:
            new_text = text[:max_seq_len]
            texts[index] = new_text

# 构造batch数据
def generate_batch(batch, pad_token_id=0, return_label=True):
    """
    Returns:
        batch(:obj:`Tuple[list]`): The batch data which contains texts, seq_lens and labels.
    """
    seq_lens = [entry[1] for entry in batch]

    batch_max_seq_len = max(seq_lens)
    texts = [entry[0] for entry in batch]
    pad_texts_to_max_seq_len(texts, batch_max_seq_len, pad_token_id)

    if return_label:
        labels = [[entry[-1]] for entry in batch]
        return texts, seq_lens, labels
    else:
        return texts, seq_lens

def create_dataloader(dataset,
                      trans_fn=None,
                      mode='train',
                      batch_size=1,
                      use_gpu=False,
                      pad_token_id=0):
    if trans_fn:
        dataset = SimpleDataset(dataset).apply(
            trans_fn, lazy=True)

    if mode == 'train' and use_gpu:
        sampler = paddle.io.DistributedBatchSampler(
            dataset=dataset, batch_size=batch_size, shuffle=True)
        dataloader = paddle.io.DataLoader(
            dataset,
            batch_sampler=sampler,
            return_list=True,
            collate_fn=lambda batch: generate_batch(batch,
                                                    pad_token_id=pad_token_id))
    else:
        shuffle = True if mode == 'train' else False
        sampler = paddle.io.BatchSampler(
            dataset=dataset, batch_size=batch_size, shuffle=shuffle)
        dataloader = paddle.io.DataLoader(
            dataset,
            batch_sampler=sampler,
            return_list=True,
            collate_fn=lambda batch: generate_batch(batch,
                                                    pad_token_id=pad_token_id))
    return dataloader

def convert_example(example, vocab, unk_token_id=1, is_test=False):
    input_ids = []
    for token in jieba.cut(example[0]):
        token_id = vocab.get(token, unk_token_id)
        input_ids.append(token_id)
    valid_length = len(input_ids)

    if not is_test:
        label = np.array(example[-1], dtype="int64")
        return input_ids, valid_length, label
    else:
        return input_ids, valid_length

1256608 words <class 'dict'>
[PAD] 0
[UNK] 1
一斤三 2
意面屋 3
11点25分 4


#### 构造batch数据集、dataloder

In [ ]:
batch_size = 64
epochs = 1
lr = 5e-4

trans_fn = partial(
        convert_example,
        vocab=vocab,
        unk_token_id=vocab['[UNK]'],
        is_test=False)

train_loader = create_dataloader(
        train_dataset,
        trans_fn=trans_fn,
        batch_size=batch_size,
        mode='train',
        pad_token_id=pad_token_id)
dev_loader = create_dataloader(
        dev_dataset,
        trans_fn=trans_fn,
        batch_size=batch_size,
        mode='validation',
        pad_token_id=pad_token_id)
test_loader = create_dataloader(
        test_dataset,
        trans_fn=trans_fn,
        batch_size=batch_size,
        mode='test',
        pad_token_id=pad_token_id)
for k in train_loader:
        print (type(k))
        # print (k)
        break


Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.792 seconds.
Prefix dict has been built successfully.


<class 'list'>


## 三、组网、训练、保存模型

### 组网
- 调用`paddle.nn.Embedding`获得词向量   
- 调用`paddlenlp.seq2vec的encoder`模型，对句子建模，得到句子向量
- 句子向量接入分类器，这里使用`paddle.nn.Linear`

In [25]:
from encoder import LSTMEncoder

class LSTMModel(nn.Layer):
    def __init__(self,
                 vocab_size,
                 num_labels,
                 emb_dim=128,
                 padding_idx=0,
                 lstm_hidden_size=198,
                 direction='forward',
                 lstm_layers=1,
                 dropout_rate=0.0,
                 pooling_type=None,
                 fc_hidden_size=96):
        super().__init__()
        self.embedder = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=emb_dim,
            padding_idx=padding_idx)
        self.lstm_encoder = LSTMEncoder(
            emb_dim,
            lstm_hidden_size,
            num_layers=lstm_layers,
            direction=direction,
            dropout=dropout_rate,
            pooling_type=pooling_type)
        self.fc = nn.Linear(self.lstm_encoder.get_output_dim(), fc_hidden_size)
        self.output_layer = nn.Linear(fc_hidden_size, num_labels)

    def forward(self, text, seq_len):
        # Shape: (batch_size, num_tokens, embedding_dim)
        # print (text.shape)
        embedded_text = self.embedder(text)
        # Shape: (batch_size, num_tokens, num_directions*lstm_hidden_size)
        # num_directions = 2 if direction is 'bidirectional'
        # if not, num_directions = 1
        text_repr = self.lstm_encoder(embedded_text, sequence_length=seq_len)
        # Shape: (batch_size, fc_hidden_size)
        fc_out = paddle.tanh(self.fc(text_repr))
        # Shape: (batch_size, num_labels)
        logits = self.output_layer(fc_out)
        return logits

- 调用`paddle.Model`封装网络


In [26]:
vocab_size = len(vocab)
num_labels = len(train_dataset.get_labels())
network = LSTMModel(
            vocab_size=vocab_size,
            num_labels=num_labels,
            direction='bidirectional',
            padding_idx=pad_token_id)
model = paddle.Model(network)

- 可调用`model.summary`查看网络结构
<font color=red size=4>求助，这里没跑通</font>

In [27]:
model.summary([(64, 200), (200)], dtype='int64')

第一层是Embedding层，将每个句子中的词转换为词向量，输出的维度是：句子个数 * 嵌入维度(20)。

接下来是LSTM层，将句子表示成向量，输出的纬度是。 

然后是全连接层(Full-connected, FC)，即Dense层，x个节点，输出的纬度是。 
 
最后一层是全连接层，只有一个节点。使用sigmoid激活函数，输出值是float，范围0-1，代表可能性/置信度。

### 网络配置、训练和保存
- 调用`model.prepare`配置模型，如损失函数、优化器。
- 调用`model.fit`训练模型。

In [75]:
optimizer = paddle.optimizer.Adam(
        parameters=model.parameters(), learning_rate=lr)

loss = paddle.nn.CrossEntropyLoss()
metric = paddle.metric.Accuracy(name="acc_accumulation")

model.prepare(optimizer, loss, metric)

model.fit(train_loader,
              dev_loader,
              epochs=epochs,
              eval_freq=1,
              log_freq=10,
              save_dir='./checkpoints',
              save_freq=1)



Epoch 1/1
step  10/150 - loss: 0.0489 - acc_accumulation: 0.9828 - 90ms/step
step  20/150 - loss: 0.0210 - acc_accumulation: 0.9844 - 82ms/step
step  30/150 - loss: 0.0107 - acc_accumulation: 0.9828 - 77ms/step
step  40/150 - loss: 0.1994 - acc_accumulation: 0.9812 - 76ms/step
step  50/150 - loss: 0.0354 - acc_accumulation: 0.9822 - 75ms/step
step  60/150 - loss: 0.0130 - acc_accumulation: 0.9820 - 74ms/step
step  70/150 - loss: 0.2209 - acc_accumulation: 0.9806 - 73ms/step
step  80/150 - loss: 0.0503 - acc_accumulation: 0.9795 - 73ms/step
step  90/150 - loss: 0.0608 - acc_accumulation: 0.9793 - 72ms/step
step 100/150 - loss: 0.1298 - acc_accumulation: 0.9794 - 72ms/step
step 110/150 - loss: 0.1620 - acc_accumulation: 0.9786 - 72ms/step
step 120/150 - loss: 0.1417 - acc_accumulation: 0.9777 - 71ms/step
step 130/150 - loss: 0.0552 - acc_accumulation: 0.9770 - 70ms/step
step 140/150 - loss: 0.1250 - acc_accumulation: 0.9770 - 70ms/step
step 150/150 - loss: 0.2011 - acc_accumulation: 0.97

- 调用`model.evaluate`评估模型

In [76]:
results = model.evaluate(test_loader)
print("Finally test acc: %.5f" % results['acc_accumulation'])

Eval begin...
step 10/19 - loss: 0.2477 - acc_accumulation: 0.8812 - 68ms/step
step 19/19 - loss: 0.1612 - acc_accumulation: 0.8992 - 50ms/step
Eval samples: 1200
Finally test acc: 0.89917


这个非常基础的模型达到了90%的正确率，复杂一点的模型可以达到x%。

## 四、网络加载和预测

- 调用`model.predict`进行预测

In [77]:
r = model.predict(test_loader)
print (type(r), len(r), len(r[0][0]))

Predict begin...
step 19/19 [==============================] - 49ms/step        
Predict samples: 1200
<class 'list'> 1 64


- 调用`model.load`加载模型

In [78]:
model_new = paddle.Model(LSTMModel(
            vocab_size=vocab_size,
            num_labels=num_labels,
            direction='bidirectional',
            padding_idx=pad_token_id))
# Loads model parameters.
params_path = 'checkpoints/final.pdparams'
model_new.load(params_path)
print("Loaded parameters from %s" % params_path)

Loaded parameters from checkpoints/final.pdparams


- 通过`model.network.eval`设置预测模式，
<font color=red size=4>预测时不要构造batch、dataloader，使用`model.predict`</font>

In [94]:
from collections import namedtuple

def preprocess_prediction_data(data):
    Example = namedtuple('Example', ['text', 'seq_len'])
    examples = []
    for text in data:
        tokens = " ".join(jieba.cut(text)).split(' ')
        ids = convert_tokens_to_ids(tokens, vocab)
        example = Example(text=ids, seq_len=len(ids))
        examples.append(example)
    return examples

def predict(model, data, label_map, collate_fn, batch_size=1):
    # Seperates data into some batches.
    batches = []
    one_batch = []
    for example in data:
        one_batch.append(example)
        if len(one_batch) == batch_size:
            batches.append(one_batch)
            one_batch = []
    if one_batch:
        batches.append(one_batch)

    results = []
    model.network.eval()
    for batch in batches:
        texts, seq_lens = collate_fn(
            batch, pad_token_id=pad_token_id, return_label=False)
        texts = paddle.to_tensor(texts)
        seq_lens = paddle.to_tensor(seq_lens)
        logits = model.network(texts, seq_lens)
        probs = F.softmax(logits, axis=-1)
        idx = paddle.argmax(probs, axis=1).numpy()
        idx = idx.tolist()
        labels = [label_map[i] for i in idx]
        results.extend(labels)
    return results

data = [
    '这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般',
    '怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片',
    '作为老的四星酒店，房间依然很整洁，相当不错。机场接机服务很好，可以在车上办理入住手续，节省时间。',
]
examples = preprocess_prediction_data(data)
print (len(examples), examples[0])
label_map = {0: 'negative', 1: 'positive'}
results = predict(
    model_new,
    examples,
    label_map=label_map,
    batch_size=batch_size,
    collate_fn=generate_batch)

for idx, text in enumerate(data):
    print('Data: {} \t Lable: {}'.format(text, results[idx]))


3 Example(text=[169379, 955944, 1108158, 276253, 823066, 1106339, 939557, 173188, 672225, 224268, 261577, 1250380, 4783, 75937, 142790, 1250380], seq_len=16)
Data: 这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般 	 Lable: negative
Data: 怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片 	 Lable: negative
Data: 作为老的四星酒店，房间依然很整洁，相当不错。机场接机服务很好，可以在车上办理入住手续，节省时间。 	 Lable: positive
